# Functions

In [59]:
def get_word_list(wordlist):
    # MUST EXECUTE fit_transform FIRST
    wordlist = vectorizer.get_feature_names()
    
    return wordlist

In [60]:
def get_bow(wordlist, df):
    # check if word exist in bow
    bow = {}
    for index, row in df.iterrows():
        txt = row['stemmed'].split()
        for i in range(len(txt)):
            if txt[i] in bow:
                bow[txt[i]] += 1
            else:
                bow[txt[i]] = 1
            
    # sort bow by value
    sorted_bow = {}
    for i in sorted(bow.items(), key=lambda x: x[1], reverse=True):
        sorted_bow[i[0]] = i[1]
        
    return sorted_bow

## Data Preprocessing

In [61]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk

df = pd.read_csv('data\data_test.csv', sep=';')

In [62]:
# Casefolding from column 'question'
df.question = df.question.str.lower()

# Remove punctuation from column 'question'
df['punc_remove'] = df.question.str.replace('[^\w\s]', ' ')

# Stemming
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()
df['stemmed'] = df.apply(lambda row: stemmer.stem(row['punc_remove']), axis = 1)

# Remove stopwords
stopword_factory = StopWordRemoverFactory()
stopword = stopword_factory.create_stop_word_remover()
df['sw_remove'] = df.apply(lambda row: stopword.remove(row['stemmed']), axis = 1)

# Tokenization
# df['tokenized'] = df.apply(lambda row: nltk.word_tokenize(row['sw_remove']), axis = 1)

## Feature Extraction

In [63]:
def flatten(term_doc_matrix):
    # Convert sparse matrix to list
    temp = []
    for i in range(term_doc_matrix.shape[0]):
        temp.append(term_doc_matrix[i].toarray().tolist())

    # Flatten list
    res = list(chain.from_iterable(temp))
    
    return res

In [64]:
def postag_weighting(tfidf, term_dict):
    # Fungsi untuk weighting tfidf based on POS Tag. Weight optimal ada di paper
    
    # cek di list tfidf, mana yg ga 0
    # indeks di list tfidf dicocokin sama indeks di term_dictionary
    # ambil nilai weight dari dict
    result = []
    
    for i in range(len(tfidf)):
        elem = []
        for j in range(len(tfidf[i])):
            if tfidf[i][j] > 0:
                temp = list(term_dict.values())[j]
                weight = 1
#                 Uncomment below if POS tagger is available
#                 if temp == 'N':
#                     weight = 3
#                 elif temp == 'ADJ':
#                     weight = 2
#                 else:
#                     weight = 1
            elem.append(tfidf[i][j] * weight)
        result.append(elem)
    return result

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import chain
import numpy as np

# Buat ngitung nilai TFIDF dari tiap term
vectorizer = TfidfVectorizer()
term_doc_matrix = vectorizer.fit_transform(df['stemmed'])

# Convert hasil fit_transform dari sparse matrix ke list
tfidf = flatten(term_doc_matrix)

# Apply weighting untuk TFIDF
# tfpos_idf = postag_weighting(tfidf, term_dict)

# Convert TFPOS-IDF ke dataframe
# Ini yang dipake buat model
df_dataset = pd.DataFrame(tfidf)
df_dataset.tail()

0    1    2    3    4    5         6    7    8    9    ...  543  544  \
166  0.0  0.0  0.0  0.0  0.0  0.0  0.183273  0.0  0.0  0.0  ...  0.0  0.0   
167  0.0  0.0  0.0  0.0  0.0  0.0  0.192530  0.0  0.0  0.0  ...  0.0  0.0   
168  0.0  0.0  0.0  0.0  0.0  0.0  0.209466  0.0  0.0  0.0  ...  0.0  0.0   
169  0.0  0.0  0.0  0.0  0.0  0.0  0.209522  0.0  0.0  0.0  ...  0.0  0.0   
170  0.0  0.0  0.0  0.0  0.0  0.0  0.204318  0.0  0.0  0.0  ...  0.0  0.0   

     545  546  547  548       549  550  551  552  
166  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  
167  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  
168  0.0  0.0  0.0  0.0  0.186082  0.0  0.0  0.0  
169  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  
170  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  

[5 rows x 553 columns]

## SVM    

In [56]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_dataset, df['Label'], test_size=0.2, random_state=23)

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall: ", metrics.recall_score(y_test, y_pred, average='weighted'))

Accuracy:  0.45714285714285713
Precision:  0.5932234432234432
Recall:  0.45714285714285713


D:\Coding\Software\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Naive Bayes

In [57]:
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(tfidf, label, test_size=0.2, random_state=23)
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
y_pred = clf_nb.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall: ", metrics.recall_score(y_test, y_pred, average='weighted'))

Accuracy:  0.4
Precision:  0.5471916971916971
Recall:  0.4


D:\Coding\Software\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Playground

In [2]:
swlist = ['a', 'ada', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 
           'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 
           'anda', 'andalah', 'antar', 'antara', 'antaranya',  'apaan', 'apabila', 
           'apalagi', 'apatah',  'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal',
           'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimanapun', 'bagi', 'bagian', 'bahkan','bahwa',
           'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah',
           'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah',
           'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah',
           'benar', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya',
           'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'berikut',
           'berikutnya', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan', 'berkenaan',
           'berlainan', 'berlalu', 'berlangsung', 'berlebihan', 'bermacam', 'bermacam-macam',
           'bermaksud', 'bermula', 'bersama', 'bersama-sama', 'bersiap', 'bersiap-siap',
           'bertanya', 'bertanya-tanya', 'berturut', 'berturut-turut', 'bertutur', 'berujar',
           'berupa', 'besar', 'betul', 'biasa', 'biasanya', 'bila', 'bilakah', 'bisa', 'boleh', 'bolehkah', 'bolehlah', 'buat',
           'bukan', 'bukankah', 'bukanlah', 'bukannya', 'bulan', 'bung', 'c', 'cara', 'caranya', 'cukup', 'cukupkah',
           'cukuplah', 'cuma', 'd', 'dahulu', 'dalam', 'dan', 'dapat', 'dari', 'daripada', 'datang', 'dekat', 'demi',
           'demikian', 'demikianlah', 'dengan', 'depan', 'di', 'dia', 'diakhiri', 'diakhirinya',
           'dialah', 'diantara', 'diberi', 'diberikan', 'diberikannya', 'dibuat', 'dibuatnya', 'didapat',
           'didatangkan', 'digunakan', 'diibaratkan', 'diibaratkannya', 'diingat', 'diingatkan', 'diinginkan',
           'dijawab', 'dijelaskan', 'dijelaskannya', 'dikarenakan', 'dikatakan', 'dikatakannya', 'dikerjakan',
           'diketahui', 'diketahuinya', 'dikira', 'dilakukan', 'dilalui', 'dilihat', 'dimaksud', 'dimaksudkan',
           'dimaksudkannya', 'dimaksudnya', 'diminta', 'dimintai', 'dimisalkan', 'dimulai', 'dimulailah',
           'dimulainya', 'dimungkinkan', 'dini', 'dipastikan', 'diperbuat', 'diperbuatnya', 'dipergunakan',
           'diperkirakan', 'diperlihatkan', 'diperlukan', 'diperlukannya', 'dipersoalkan', 'dipertanyakan',
           'dipunyai', 'diri', 'dirinya', 'disampaikan', 'disebutkan', 'disebutkannya', 'disini', 'disinilah',
           'ditambahkan', 'ditandaskan', 'ditanya', 'ditanyai', 'ditanyakan', 'ditegaskan', 'ditujukan', 'ditunjuk',
           'ditunjuki', 'ditunjukkan', 'ditunjukkannya', 'ditunjuknya', 'dituturkan', 'dituturkannya', 'diucapkan',
           'diucapkannya', 'diungkapkan', 'dong', 'dua', 'dulu', 'e', 'empat', 'enak', 'enggak', 'enggaknya',
           'entah', 'entahlah', 'f', 'g', 'guna', 'gunakan', 'h', 'hadap', 'hai', 'hal', 'halo', 'hallo',
           'hampir', 'hanya', 'hanyalah', 'hari', 'harus', 'haruslah', 'harusnya', 'helo', 'hello', 'hendak',
           'hendaklah', 'hendaknya', 'hingga', 'i', 'ia', 'ialah', 'ibarat', 'ibaratkan', 'ibaratnya', 'ibu', 'ikut',
           'ingat', 'ingat-ingat', 'ingin', 'inginkah', 'inginkan', 'ini', 'inikah', 'inilah', 'itu', 'itukah', 'itulah',
           'j', 'jadi', 'jadilah', 'jadinya', 'jangan', 'jangankan', 'janganlah', 'jauh', 'jawab', 'jawaban',
           'jawabnya', 'jelas', 'jelaslah', 'jelasnya', 'jika', 'jikalau', 'juga', 'jumlah', 'jumlahnya', 'justru',
           'k', 'kadar', 'kala', 'kalau', 'kalaulah', 'kalaupun', 'kali', 'kalian', 'kami', 'kamilah', 'kamu',
           'kamulah', 'kan', 'kapankah', 'kapanpun',  'karenanya', 'kasus', 'kata', 'katakan', 'katakanlah', 'katanya',
           'ke', 'keadaan', 'kebetulan', 'kecil', 'kedua', 'keduanya', 'keinginan', 'kelamaan', 'kelihatan',
           'kelihatannya', 'kelima', 'keluar', 'kembali', 'kemudian', 'kemungkinan', 'kemungkinannya', 'kena',
           'kepada', 'kepadanya', 'kerja', 'kesampaian', 'keseluruhan', 'keseluruhannya', 'keterlaluan', 'ketika',
           'khusus', 'khususnya', 'kini', 'kinilah', 'kira', 'kira-kira', 'kiranya', 'kita', 'kitalah', 'kok', 'kurang',
           'l', 'lagi', 'lagian', 'lah', 'lain', 'lainnya', 'laku', 'lalu', 'lama', 'lamanya', 'langsung', 'lanjut',
           'lanjutnya', 'lebih', 'lewat', 'lihat', 'lima', 'luar', 'm', 'macam', 'maka', 'makanya', 'makin', 'maksud',
           'malah', 'malahan', 'mampu', 'mampukah', 'mana', 'manakala', 'manalagi', 'masa', 'masalah', 'masalahnya',
           'masih', 'masihkah', 'masing', 'masing-masing', 'masuk', 'mata', 'mau', 'maupun', 'melainkan', 'melakukan',
           'melalui', 'melihat', 'melihatnya', 'memang', 'memastikan', 'memberi', 'memberikan', 'membuat',
           'memerlukan', 'memihak', 'meminta', 'memintakan', 'memisalkan', 'memperbuat', 'mempergunakan',
           'memperkirakan', 'memperlihatkan', 'mempersiapkan', 'mempersoalkan', 'mempertanyakan', 'mempunyai',
           'memulai', 'memungkinkan', 'menaiki', 'menambahkan', 'menandaskan', 'menanti', 'menanti-nanti',
           'menantikan', 'menanya', 'menanyai', 'menanyakan', 'mendapat', 'mendapatkan', 'mendatang', 'mendatangi',
           'mendatangkan', 'menegaskan', 'mengakhiri', 'mengatakan', 'mengatakannya', 'mengenai', 'mengerjakan',
           'mengetahui', 'menggunakan', 'menghendaki', 'mengibaratkan', 'mengibaratkannya', 'mengingat', 'mengingatkan',
           'menginginkan', 'mengira', 'mengucapkan', 'mengucapkannya', 'mengungkapkan', 'menjadi', 'menjawab',
           'menjelaskan', 'menuju', 'menunjuk', 'menunjuki', 'menunjuknya', 'menurut', 'menuturkan', 'menyampaikan',
           'menyangkut', 'menyatakan', 'menyebutkan', 'menyeluruh', 'menyiapkan', 'merasa', 'mereka', 'merekalah',
           'meski', 'meskipun', 'meyakini', 'meyakinkan', 'minta', 'mirip', 'misal', 'misalkan', 'misalnya',
           'mohon', 'mula', 'mulai', 'mulailah', 'mulanya', 'mungkin', 'mungkinkah', 'n', 'nah', 'naik', 'namun',
           'nanti', 'nantinya', 'nya', 'nyaris', 'nyata', 'nyatanya', 'o', 'oleh', 'olehnya', 'orang', 'p', 'pada',
           'padahal', 'padanya', 'pak', 'paling', 'panjang', 'pantas', 'para', 'pasti', 'pastilah', 'penting',
           'pentingnya', 'per', 'percuma', 'perlu', 'perlukah', 'perlunya', 'pernah', 'persoalan', 'pertama',
           'pertama-tama', 'pertanyaan', 'pertanyakan', 'pihak', 'pihaknya', 'pukul', 'pula', 'pun', 'punya', 'q', 'r',
           'rasa', 'rasanya', 'rupanya', 's', 'saat', 'saatnya', 'saja', 'sajalah', 'salam', 'saling', 'sama',
           'sama-sama', 'sambil', 'sampai', 'sampai-sampai', 'sampaikan', 'sana', 'sangat', 'sangatlah', 'sangkut',
           'satu', 'saya', 'sayalah', 'se', 'sebab', 'sebabnya', 'sebagai', 'sebagaimana', 'sebagainya', 'sebagian',
           'sebaik', 'sebaik-baiknya', 'sebaiknya', 'sebaliknya', 'sebanyak', 'sebegini', 'sebegitu', 'sebelum',
           'sebelumnya', 'sebenarnya', 'seberapa', 'sebesar', 'sebetulnya', 'sebisanya', 'sebuah', 
           'sebutnya', 'secara', 'secukupnya', 'sedang', 'sedangkan', 'sedemikian', 'sedikit', 'sedikitnya', 'seenaknya',
           'segala', 'segalanya', 'segera', 'seharusnya', 'sehingga', 'seingat', 'sejak', 'sejauh', 'sejenak', 'sejumlah',
           'sekadar', 'sekadarnya', 'sekali', 'sekali-kali', 'sekalian', 'sekaligus', 'sekalipun', 'sekarang', 'sekaranglah',
           'sekecil', 'seketika', 'sekiranya', 'sekitar', 'sekitarnya', 'sekurang-kurangnya', 'sekurangnya', 'sela',
           'selain', 'selaku', 'selalu', 'selama', 'selama-lamanya', 'selamanya', 'selanjutnya', 'seluruh', 'seluruhnya',
           'semacam', 'semakin', 'semampu', 'semampunya', 'semasa', 'semasih', 'semata', 'semata-mata', 'semaunya',
           'sementara', 'semisal', 'semisalnya', 'sempat', 'semua', 'semuanya', 'semula', 'sendiri', 'sendirian',
           'sendirinya', 'seolah', 'seolah-olah', 'seorang', 'sepanjang', 'sepantasnya', 'sepantasnyalah', 'seperlunya',
           'seperti', 'sepertinya', 'sepihak', 'sering', 'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai',
           'sesegera', 'sesekali', 'seseorang', 'sesuatu', 'sesuatunya', 'sesudah', 'sesudahnya', 'setelah', 'setempat',
           'setengah', 'seterusnya', 'setiap', 'setiba', 'setibanya', 'setidak-tidaknya', 'setidaknya', 'setinggi', 'seusai',
           'sewaktu', 'siap', 'siapa', 'siapakah', 'siapapun', 'sini', 'sinilah', 'soal', 'soalnya', 'suatu', 'sudah', 
           'sudahkah', 'sudahlah', 'supaya', 't', 'tadi', 'tadinya', 'tahu', 'tak', 'tambah', 'tambahnya', 'tampak',
           'tampaknya', 'tandas', 'tandasnya', 'tanpa', 'tanya', 'tanyakan', 'tanyanya', 'tapi', 'tegas', 'tegasnya',
           'telah', 'tempat', 'tentang', 'tentu', 'tentulah', 'tentunya', 'tepat', 'terakhir', 'terasa', 'terbanyak',
           'terdahulu', 'terdapat', 'terdiri', 'terhadap', 'terhadapnya', 'teringat', 'teringat-ingat', 'terjadi',
           'terjadilah', 'terjadinya', 'terkira', 'terlalu', 'terlebih', 'terlihat', 'termasuk', 'ternyata', 'tersampaikan',
           'tersebut', 'tersebutlah', 'tertentu', 'tertuju', 'terus', 'terutama', 'tetap', 'tetapi', 'tiap', 'tiba',
           'tiba-tiba', 'tidak', 'tidakkah', 'tidaklah', 'tiga', 'toh', 'tuju', 'tunjuk', 'turut', 'tutur', 'tuturnya',
           'u', 'ucap', 'ucapnya', 'ujar', 'ujarnya', 'umumnya', 'ungkap', 'ungkapnya', 'untuk', 'usah', 'usai', 'v', 'w',
           'waduh', 'wah', 'wahai', 'waktunya', 'walau', 'walaupun', 'wong', 'x', 'y', 'ya', 'yaitu', 'yakin', 'yakni',
           'yang', 'z', '']

In [3]:
def remove_stopwords(text):
    res = []
    words = text.split(" ")
    for word in words:
        if word not in swlist:
            res.append(word)
    return ' '.join(res)

In [4]:
sw_keep = ['adalah', 'apa', 'apakah', 'arti', 'artinya', 'bagaimana', 'bagaimanakah', 'berapa', 'berapakah', 'benarkah',
           'beri', 'berikan', 'berjumlah', 'betulkah', 'bisakah', 'diantaranya', 'disebut', 'jelaskan', 'kapan',
           'karena', 'kenapa', 'mengapa', 'menunjukkan', 'merupakan', 'rupa', 'sebut', 'sebutlah']
    
df = pd.read_csv('data\data_test.csv', sep=';')

In [5]:
# Casefolding from column 'question'
df.question = df.question.str.lower()

# Remove punctuation from column 'question'
df['punc_remove'] = df.question.str.replace('[^\w\s]', ' ')

# Remove stopwords
df['sw_remove'] = df.apply(lambda row: remove_stopwords(row['punc_remove']), axis = 1) 

# Stemming
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()
df['stemmed'] = df.apply(lambda row: stemmer.stem(row['sw_remove']), axis = 1)

# POSTagger
# Pake program dari bu ade

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

corpus = ['this is the first document',
          'this document is the second document',
          'and this is the third one',
          'is this the first document']
vocabulary = ['this', 'document', 'first', 'is', 'second', 'the', 'and', 'one']

pipe = Pipeline([
                 ('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfidf', TfidfTransformer())
                ]
               ).fit(corpus)

pipe['count'].transform(corpus).toarray()

array([[1, 1, 1, 1, 0, 1, 0, 0],
       [1, 2, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 0, 0]], dtype=int64)

In [22]:
x = pipe['tfidf'].idf_
print(x)
print(len(x))

[1.         1.22314355 1.51082562 1.         1.91629073 1.
 1.91629073 1.91629073]
8


In [46]:
pipe['count'].get_feature_names_out(corpus)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names_out'

In [38]:
print(pipe.transform(corpus))

  (0, 5)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045
  (0, 0)	0.38408524091481483
  (1, 5)	0.281088674033753
  (1, 4)	0.5386476208856763
  (1, 3)	0.281088674033753
  (1, 1)	0.6876235979836938
  (1, 0)	0.281088674033753
  (2, 7)	0.5958130328252973
  (2, 6)	0.5958130328252973
  (2, 5)	0.3109199574547778
  (2, 3)	0.3109199574547778
  (2, 0)	0.3109199574547778
  (3, 5)	0.38408524091481483
  (3, 3)	0.38408524091481483
  (3, 2)	0.5802858236844359
  (3, 1)	0.46979138557992045
  (3, 0)	0.38408524091481483


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
dataset = vectorizer.fit_transform(df['stemmed'])
label = df['Label']

In [53]:
vectorizer.get_feature_names()

['12jalan',
 '195',
 '27',
 '44',
 'acak',
 'adalah',
 'adik',
 'aja',
 'aktif',
 'alam',
 'alat',
 'amanat',
 'ana',
 'anak',
 'andal',
 'andika',
 'anggap',
 'anggota',
 'angin',
 'antara',
 'apa',
 'apakah',
 'arah',
 'arang',
 'arti',
 'asa',
 'asal',
 'aspek',
 'asyik',
 'atas',
 'awet',
 'ayam',
 'baca',
 'bagaimana',
 'bagus',
 'bahan',
 'bahas',
 'bahasa',
 'baik',
 'balas',
 'bandung',
 'bangga',
 'bantu',
 'barang',
 'barat',
 'baris',
 'barung',
 'batok',
 'belah',
 'benam',
 'benang',
 'bendera',
 'bentuk',
 'berani',
 'beruang',
 'biar',
 'biasa',
 'bicara',
 'bidang',
 'bilal',
 'binatang',
 'boros',
 'buah',
 'buat',
 'buka',
 'buku',
 'bunga',
 'bunyi',
 'buruk',
 'burung',
 'cakap',
 'cari',
 'cepat',
 'cerah',
 'cerdik',
 'cerita',
 'cermat',
 'cetak',
 'ciri',
 'citra',
 'ciwidey',
 'contoh',
 'cuci',
 'cuplik',
 'daerah',
 'dagu',
 'darat',
 'dari',
 'dasar',
 'data',
 'deli',
 'denah',
 'dengan',
 'dengar',
 'desis',
 'deskripsi',
 'di',
 'diam',
 'dinas',
 'dingin

In [54]:
vectorizer.idf_

array([5.4543473 , 5.4543473 , 5.4543473 , 5.4543473 , 5.4543473 ,
       1.83000636, 5.4543473 , 5.4543473 , 5.04888219, 5.4543473 ,
       4.20158433, 5.4543473 , 5.4543473 , 5.04888219, 5.4543473 ,
       5.4543473 , 5.4543473 , 5.4543473 , 5.4543473 , 5.04888219,
       4.06805294, 5.04888219, 5.4543473 , 5.4543473 , 4.76120012,
       5.4543473 , 5.4543473 , 5.4543473 , 5.04888219, 5.4543473 ,
       5.4543473 , 5.4543473 , 4.20158433, 5.4543473 , 5.4543473 ,
       4.53805656, 5.4543473 , 4.53805656, 4.76120012, 5.04888219,
       5.04888219, 5.4543473 , 5.4543473 , 5.04888219, 5.4543473 ,
       5.04888219, 5.4543473 , 5.4543473 , 5.04888219, 5.4543473 ,
       5.4543473 , 5.4543473 , 4.53805656, 4.76120012, 5.4543473 ,
       5.4543473 , 5.04888219, 5.4543473 , 5.04888219, 5.4543473 ,
       5.04888219, 5.4543473 , 5.04888219, 4.53805656, 5.4543473 ,
       4.53805656, 5.4543473 , 5.4543473 , 5.04888219, 5.4543473 ,
       4.76120012, 5.4543473 , 5.4543473 , 4.76120012, 5.45434

In [ ]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=23)

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall: ", metrics.recall_score(y_test, y_pred, average='weighted'))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=23)
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
y_pred = clf_nb.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall: ", metrics.recall_score(y_test, y_pred, average='weighted'))

In [ ]:
swlist = ['a', 'ada', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 
           'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 
           'anda', 'andalah', 'antar', 'antara', 'antaranya',  'apaan', 'apabila', 
           'apalagi', 'apatah',  'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal',
           'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimanapun', 'bagi', 'bagian', 'bahkan','bahwa',
           'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah',
           'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah',
           'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah',
           'benar', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya',
           'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'berikut',
           'berikutnya', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan', 'berkenaan',
           'berlainan', 'berlalu', 'berlangsung', 'berlebihan', 'bermacam', 'bermacam-macam',
           'bermaksud', 'bermula', 'bersama', 'bersama-sama', 'bersiap', 'bersiap-siap',
           'bertanya', 'bertanya-tanya', 'berturut', 'berturut-turut', 'bertutur', 'berujar',
           'berupa', 'besar', 'betul', 'biasa', 'biasanya', 'bila', 'bilakah', 'bisa', 'boleh', 'bolehkah', 'bolehlah', 'buat',
           'bukan', 'bukankah', 'bukanlah', 'bukannya', 'bulan', 'bung', 'c', 'cara', 'caranya', 'cukup', 'cukupkah',
           'cukuplah', 'cuma', 'd', 'dahulu', 'dalam', 'dan', 'dapat', 'dari', 'daripada', 'datang', 'dekat', 'demi',
           'demikian', 'demikianlah', 'dengan', 'depan', 'di', 'dia', 'diakhiri', 'diakhirinya',
           'dialah', 'diantara', 'diberi', 'diberikan', 'diberikannya', 'dibuat', 'dibuatnya', 'didapat',
           'didatangkan', 'digunakan', 'diibaratkan', 'diibaratkannya', 'diingat', 'diingatkan', 'diinginkan',
           'dijawab', 'dijelaskan', 'dijelaskannya', 'dikarenakan', 'dikatakan', 'dikatakannya', 'dikerjakan',
           'diketahui', 'diketahuinya', 'dikira', 'dilakukan', 'dilalui', 'dilihat', 'dimaksud', 'dimaksudkan',
           'dimaksudkannya', 'dimaksudnya', 'diminta', 'dimintai', 'dimisalkan', 'dimulai', 'dimulailah',
           'dimulainya', 'dimungkinkan', 'dini', 'dipastikan', 'diperbuat', 'diperbuatnya', 'dipergunakan',
           'diperkirakan', 'diperlihatkan', 'diperlukan', 'diperlukannya', 'dipersoalkan', 'dipertanyakan',
           'dipunyai', 'diri', 'dirinya', 'disampaikan', 'disebutkan', 'disebutkannya', 'disini', 'disinilah',
           'ditambahkan', 'ditandaskan', 'ditanya', 'ditanyai', 'ditanyakan', 'ditegaskan', 'ditujukan', 'ditunjuk',
           'ditunjuki', 'ditunjukkan', 'ditunjukkannya', 'ditunjuknya', 'dituturkan', 'dituturkannya', 'diucapkan',
           'diucapkannya', 'diungkapkan', 'dong', 'dua', 'dulu', 'e', 'empat', 'enak', 'enggak', 'enggaknya',
           'entah', 'entahlah', 'f', 'g', 'guna', 'gunakan', 'h', 'hadap', 'hai', 'hal', 'halo', 'hallo',
           'hampir', 'hanya', 'hanyalah', 'hari', 'harus', 'haruslah', 'harusnya', 'helo', 'hello', 'hendak',
           'hendaklah', 'hendaknya', 'hingga', 'i', 'ia', 'ialah', 'ibarat', 'ibaratkan', 'ibaratnya', 'ibu', 'ikut',
           'ingat', 'ingat-ingat', 'ingin', 'inginkah', 'inginkan', 'ini', 'inikah', 'inilah', 'itu', 'itukah', 'itulah',
           'j', 'jadi', 'jadilah', 'jadinya', 'jangan', 'jangankan', 'janganlah', 'jauh', 'jawab', 'jawaban',
           'jawabnya', 'jelas', 'jelaslah', 'jelasnya', 'jika', 'jikalau', 'juga', 'jumlah', 'jumlahnya', 'justru',
           'k', 'kadar', 'kala', 'kalau', 'kalaulah', 'kalaupun', 'kali', 'kalian', 'kami', 'kamilah', 'kamu',
           'kamulah', 'kan', 'kapankah', 'kapanpun',  'karenanya', 'kasus', 'kata', 'katakan', 'katakanlah', 'katanya',
           'ke', 'keadaan', 'kebetulan', 'kecil', 'kedua', 'keduanya', 'keinginan', 'kelamaan', 'kelihatan',
           'kelihatannya', 'kelima', 'keluar', 'kembali', 'kemudian', 'kemungkinan', 'kemungkinannya', 'kena',
           'kepada', 'kepadanya', 'kerja', 'kesampaian', 'keseluruhan', 'keseluruhannya', 'keterlaluan', 'ketika',
           'khusus', 'khususnya', 'kini', 'kinilah', 'kira', 'kira-kira', 'kiranya', 'kita', 'kitalah', 'kok', 'kurang',
           'l', 'lagi', 'lagian', 'lah', 'lain', 'lainnya', 'laku', 'lalu', 'lama', 'lamanya', 'langsung', 'lanjut',
           'lanjutnya', 'lebih', 'lewat', 'lihat', 'lima', 'luar', 'm', 'macam', 'maka', 'makanya', 'makin', 'maksud',
           'malah', 'malahan', 'mampu', 'mampukah', 'mana', 'manakala', 'manalagi', 'masa', 'masalah', 'masalahnya',
           'masih', 'masihkah', 'masing', 'masing-masing', 'masuk', 'mata', 'mau', 'maupun', 'melainkan', 'melakukan',
           'melalui', 'melihat', 'melihatnya', 'memang', 'memastikan', 'memberi', 'memberikan', 'membuat',
           'memerlukan', 'memihak', 'meminta', 'memintakan', 'memisalkan', 'memperbuat', 'mempergunakan',
           'memperkirakan', 'memperlihatkan', 'mempersiapkan', 'mempersoalkan', 'mempertanyakan', 'mempunyai',
           'memulai', 'memungkinkan', 'menaiki', 'menambahkan', 'menandaskan', 'menanti', 'menanti-nanti',
           'menantikan', 'menanya', 'menanyai', 'menanyakan', 'mendapat', 'mendapatkan', 'mendatang', 'mendatangi',
           'mendatangkan', 'menegaskan', 'mengakhiri', 'mengatakan', 'mengatakannya', 'mengenai', 'mengerjakan',
           'mengetahui', 'menggunakan', 'menghendaki', 'mengibaratkan', 'mengibaratkannya', 'mengingat', 'mengingatkan',
           'menginginkan', 'mengira', 'mengucapkan', 'mengucapkannya', 'mengungkapkan', 'menjadi', 'menjawab',
           'menjelaskan', 'menuju', 'menunjuk', 'menunjuki', 'menunjuknya', 'menurut', 'menuturkan', 'menyampaikan',
           'menyangkut', 'menyatakan', 'menyebutkan', 'menyeluruh', 'menyiapkan', 'merasa', 'mereka', 'merekalah',
           'meski', 'meskipun', 'meyakini', 'meyakinkan', 'minta', 'mirip', 'misal', 'misalkan', 'misalnya',
           'mohon', 'mula', 'mulai', 'mulailah', 'mulanya', 'mungkin', 'mungkinkah', 'n', 'nah', 'naik', 'namun',
           'nanti', 'nantinya', 'nya', 'nyaris', 'nyata', 'nyatanya', 'o', 'oleh', 'olehnya', 'orang', 'p', 'pada',
           'padahal', 'padanya', 'pak', 'paling', 'panjang', 'pantas', 'para', 'pasti', 'pastilah', 'penting',
           'pentingnya', 'per', 'percuma', 'perlu', 'perlukah', 'perlunya', 'pernah', 'persoalan', 'pertama',
           'pertama-tama', 'pertanyaan', 'pertanyakan', 'pihak', 'pihaknya', 'pukul', 'pula', 'pun', 'punya', 'q', 'r',
           'rasa', 'rasanya', 'rupanya', 's', 'saat', 'saatnya', 'saja', 'sajalah', 'salam', 'saling', 'sama',
           'sama-sama', 'sambil', 'sampai', 'sampai-sampai', 'sampaikan', 'sana', 'sangat', 'sangatlah', 'sangkut',
           'satu', 'saya', 'sayalah', 'se', 'sebab', 'sebabnya', 'sebagai', 'sebagaimana', 'sebagainya', 'sebagian',
           'sebaik', 'sebaik-baiknya', 'sebaiknya', 'sebaliknya', 'sebanyak', 'sebegini', 'sebegitu', 'sebelum',
           'sebelumnya', 'sebenarnya', 'seberapa', 'sebesar', 'sebetulnya', 'sebisanya', 'sebuah', 
           'sebutnya', 'secara', 'secukupnya', 'sedang', 'sedangkan', 'sedemikian', 'sedikit', 'sedikitnya', 'seenaknya',
           'segala', 'segalanya', 'segera', 'seharusnya', 'sehingga', 'seingat', 'sejak', 'sejauh', 'sejenak', 'sejumlah',
           'sekadar', 'sekadarnya', 'sekali', 'sekali-kali', 'sekalian', 'sekaligus', 'sekalipun', 'sekarang', 'sekaranglah',
           'sekecil', 'seketika', 'sekiranya', 'sekitar', 'sekitarnya', 'sekurang-kurangnya', 'sekurangnya', 'sela',
           'selain', 'selaku', 'selalu', 'selama', 'selama-lamanya', 'selamanya', 'selanjutnya', 'seluruh', 'seluruhnya',
           'semacam', 'semakin', 'semampu', 'semampunya', 'semasa', 'semasih', 'semata', 'semata-mata', 'semaunya',
           'sementara', 'semisal', 'semisalnya', 'sempat', 'semua', 'semuanya', 'semula', 'sendiri', 'sendirian',
           'sendirinya', 'seolah', 'seolah-olah', 'seorang', 'sepanjang', 'sepantasnya', 'sepantasnyalah', 'seperlunya',
           'seperti', 'sepertinya', 'sepihak', 'sering', 'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai',
           'sesegera', 'sesekali', 'seseorang', 'sesuatu', 'sesuatunya', 'sesudah', 'sesudahnya', 'setelah', 'setempat',
           'setengah', 'seterusnya', 'setiap', 'setiba', 'setibanya', 'setidak-tidaknya', 'setidaknya', 'setinggi', 'seusai',
           'sewaktu', 'siap', 'siapa', 'siapakah', 'siapapun', 'sini', 'sinilah', 'soal', 'soalnya', 'suatu', 'sudah', 
           'sudahkah', 'sudahlah', 'supaya', 't', 'tadi', 'tadinya', 'tahu', 'tak', 'tambah', 'tambahnya', 'tampak',
           'tampaknya', 'tandas', 'tandasnya', 'tanpa', 'tanya', 'tanyakan', 'tanyanya', 'tapi', 'tegas', 'tegasnya',
           'telah', 'tempat', 'tentang', 'tentu', 'tentulah', 'tentunya', 'tepat', 'terakhir', 'terasa', 'terbanyak',
           'terdahulu', 'terdapat', 'terdiri', 'terhadap', 'terhadapnya', 'teringat', 'teringat-ingat', 'terjadi',
           'terjadilah', 'terjadinya', 'terkira', 'terlalu', 'terlebih', 'terlihat', 'termasuk', 'ternyata', 'tersampaikan',
           'tersebut', 'tersebutlah', 'tertentu', 'tertuju', 'terus', 'terutama', 'tetap', 'tetapi', 'tiap', 'tiba',
           'tiba-tiba', 'tidak', 'tidakkah', 'tidaklah', 'tiga', 'toh', 'tuju', 'tunjuk', 'turut', 'tutur', 'tuturnya',
           'u', 'ucap', 'ucapnya', 'ujar', 'ujarnya', 'umumnya', 'ungkap', 'ungkapnya', 'untuk', 'usah', 'usai', 'v', 'w',
           'waduh', 'wah', 'wahai', 'waktunya', 'walau', 'walaupun', 'wong', 'x', 'y', 'ya', 'yaitu', 'yakin', 'yakni',
           'yang', 'z']


dihapus = ['adalah', 'apa', 'apakah', 'arti', 'artinya', 'bagaimana', 'bagaimanakah', 'berapa', 'berapakah', 'benarkah',
           'beri', 'berikan', 'berjumlah', 'betulkah', 'bisakah', 'bolehkah', 'diantaranya', 'disebut', 'jelaskan', 'kapan',
           'karena', 'kenapa', 'menunjukkan', 'merupakan', 'rupa', 'sebut', 'sebutlah']